In [1]:
#%%
import numpy as np
import netCDF4 as nc
import os
import re
import pandas as pd
import glob
import rasterio
from datetime import datetime, timedelta
import pandas as pd
import math

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap,BoundaryNorm
import matplotlib.cm as cm
from matplotlib import colors
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
# import imageio
# import pygeodesy
# from pygeodesy.ellipsoidalKarney import LatLon
import folium
# from shapely.geometry import Point,Polygon
# from folium.plugins import FastMarkerCluster

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.ticker as mticker
from cartopy.io import shapereader
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from scipy.signal import correlate
import matplotlib.colors as mcolors

import sys
params_path = '/data42/yingying/HydroDA/src'
#external python codes
sys.path.append(params_path)
import params_long01 as pm    # out_msm_round*
from matplotlib.patches import Rectangle


KeyboardInterrupt: 

In [ ]:
camadir  = pm.CaMa_dir()
inputdir = pm.plot_dir()
expname  = pm.runname(pm.mode())
dahour   = pm.dahour()
dahour_str = '{:02d}'.format(dahour)
exp_plotdir = inputdir + '/exp_' + expname + dahour_str + '/'
obsdir = pm.obs_dir(pm.real_mode())

# simulation start from 20191007 but analysis start from 20191010
# simulation start from 20240120 but analysis start from 20240201
start_year,start_month,start_date,start_hour=pm.starttime() # Start year month date
syyyy='%04d' % (start_year)
if syyyy == '2019':
    tstart = int(24*3)
    tstart_da = int(24*3/dahour)
elif syyyy == '2024':
    tstart = int(24*12)
    tstart_da = int(24*12/dahour)
else:
    tstart = 0
    tstart_da = 0
smm='%02d' % (start_month)
sdd='%02d' % (start_date)
shh='%02d' % (start_hour)
stime = datetime(start_year,start_month,start_date,start_hour)
end_year,end_month,end_date,end_hour=pm.endtime() # Start year month date
eyyyy='%04d' % (end_year)
emm='%02d' % (end_month)
edd='%02d' % (end_date)
ehh='%02d' % (end_hour)
etime = datetime(end_year,end_month,end_date,end_hour)
time_step = timedelta(hours=dahour)
time_len = int((etime-stime).total_seconds() / 3600)
time_len_da = int((etime-stime).total_seconds() / 3600 / dahour)

path_alloc   = pm.alloc_dir()
name_alloc   = '/wlv_2019h.xlsx'
df     = pd.read_excel(path_alloc+name_alloc)
df.rename(columns={df.columns[1]: 'latitude', df.columns[2]: 'longitude'}, inplace=True)
id_info =  np.array(df.iloc[:,0])
lat_obs =  np.array(df.iloc[:,1])
lon_obs =  np.array(df.iloc[:,2])
ix     = np.array(df.iloc[:,8])-1  # lon_ind
iy     = np.array(df.iloc[:,9])-1  # lat_ind

def read_bin2d(fname):
    with open(fname, 'rb') as file:
        data_bin = np.fromfile(file, dtype=np.float32)
        data_re  = np.reshape(data_bin,(1320,1500))
    return data_re
def read_bin3d(fname,layer):
    with open(fname, 'rb') as file:
        data_bin = np.fromfile(file, dtype=np.float32)
        data_re  = np.reshape(data_bin,(layer,1320,1500))
    return data_re
def read_bin_int(fname,layer):
    with open(fname, 'rb') as file:
        data_bin = np.fromfile(file, dtype=np.int32)
        data_re  = np.reshape(data_bin,(layer,1320,1500))
    return data_re
def read_bin_station(fname):
    with open(fname, 'rb') as file:
        data_bin = np.fromfile(file, dtype=np.float32)
    return data_bin
def read_nc(fname,layer):
    nf = nc.Dataset(fname,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    lat = np.array(nf.variables[varname[1]][:])
    lon = np.array(nf.variables[varname[2]][:])
    var = np.array(nf.variables[varname[layer]][:])
    var = np.array(np.where(var>10**7,np.nan,var))
    return var   

In [ ]:
# from 2019100100
def read_prep(file):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lon = np.array(nf.variables[varname[1]][:])
    lat = np.array(nf.variables[varname[2]][:])
    var = np.array(nf.variables[varname[3]][:])
    var = var*3600 # unit: mm/h 
    var = np.array(np.where(var<-900,np.nan,var))
    return lat,lon,var
lat,lon,rainf = read_prep('/data42/yingying/data/matsiro_frc/tej_amedas2019.nc')

In [ ]:
lat_min,lat_max,lon_min,lon_max = 30,46,128,149
def read_plot_tif(filename,lat_min,lat_max,lon_min,lon_max):
    up_area  = np.zeros(((lat_max-lat_min)*3600,(lon_max-lon_min)*3600)) 
    lat_all  = np.zeros((lat_max-lat_min)*3600) 
    lon_all  = np.zeros((lon_max-lon_min)*3600)
    filepath = pm.DA_dir()+'/src/plot/file/map/'+filename
    tiff_files = glob.glob(os.path.join(filepath, '*.tif'))
    for filenum in range(0,len(tiff_files)):
        filen = tiff_files[filenum]
        file_lon = int(filen[51:54])
        file_lat = int(filen[48:50])
        if (file_lat-lat_min>=0) &(file_lon-lon_min>=0) &(file_lat-lat_max<0) &(file_lon-lon_max<0):           
            with rasterio.open(filen) as dataset:
                tiff_data = dataset.read(1)
                # be care !! when drawing the figure should convert latitude tiff[::-1,:]
                up_area[(file_lat-lat_min)*3600:(file_lat-lat_min+1)*3600,(file_lon-lon_min)*3600:(file_lon-lon_min+1)*3600] = tiff_data[::-1,:]
    return up_area
up_plot = read_plot_tif('upa',30,46,128,149)
#%% lower resolution
size=120
uparea_filter = np.where(up_plot<10**2,np.nan,up_plot)
def low_resolution(var,size):
    var_low = np.full((int((lat_max-lat_min)*3600/size),int((lon_max-lon_min)*3600/size)),np.nan)
    for row in range(0,np.shape(var_low)[0]):
        if (np.all(np.isnan(var[row*size:(row+1)*size,:]))==1):
            continue
        for col in range(0,np.shape(var_low)[1]):
            var_low[row,col] = np.nanmean(var[row*size:(row+1)*size,col*size:(col+1)*size])
    return var_low    
up_low = low_resolution(uparea_filter,size) # for figure ploting
lon_map_low = np.linspace(lon_min,lon_max,int((lon_max-lon_min)*3600/size))
lat_map_low = np.linspace(lat_min,lat_max,int((lat_max-lat_min)*3600/size))
lon2d_low_map, lat2d_low_map = np.meshgrid(lon_map_low, lat_map_low)

# #%% read small tiff data
lat_min,lat_max,lon_min,lon_max = 34,37,137,141
up_area = read_plot_tif('upa',34,37,137,141)
#%% lower resolution
size=10
def low_resolution(var,size):
    var_low = np.full((int((lat_max-lat_min)*3600/size),int((lon_max-lon_min)*3600/size)),np.nan)
    for row in range(0,np.shape(var_low)[0]):
        if (np.all(np.isnan(var[row*size:(row+1)*size,:]))==1):
            continue
        for col in range(0,np.shape(var_low)[1]):
            var_low[row,col] = np.nanmean(var[row*size:(row+1)*size,col*size:(col+1)*size])
    return var_low    
uparea_low = low_resolution(up_area,size)
uparea_low = np.where(uparea_low<10,np.nan,uparea_low)
lon_tiff = np.linspace(lon_min,lon_max,int((lon_max-lon_min)*3600/size))
lat_tiff = np.linspace(lat_min,lat_max,int((lat_max-lat_min)*3600/size))


In [ ]:
#%% show the validation stations
# obs_situ_dir = pm.alloc_dir() + '/'
# 30 validation stations
pm_path = pm.__file__
obs_situ_dir = '/data42/yingying/HydroDA/Empirical_LocalPatch/MLIT/alloc/'
if 'round' in pm_path:
    file_select_lon = 'lon_select_index'+pm_path[-4]+'.txt'
    file_select_lat = 'lat_select_index'+pm_path[-4]+'.txt'
    file_valid_lon = 'lon_exclude_index'+pm_path[-4]+'.txt'
    file_valid_lat = 'lat_exclude_index'+pm_path[-4]+'.txt'
else:
    file_select_lon = 'lon_select_index0.txt'
    file_select_lat = 'lat_select_index0.txt'
    file_valid_lon = 'lon_exclude_index0.txt'
    file_valid_lat = 'lat_exclude_index0.txt'

lony_select = np.array(np.loadtxt(obs_situ_dir+file_select_lon),dtype=int)
latx_select = np.array(np.loadtxt(obs_situ_dir+file_select_lat),dtype=int)
lony_valid = np.array(np.loadtxt(obs_situ_dir+file_valid_lon),dtype=int)
latx_valid = np.array(np.loadtxt(obs_situ_dir+file_valid_lat),dtype=int)
lat_select  = lat[latx_select-1]
lon_select  = lon[lony_select-1]
lat_valid  = lat[latx_valid-1]
lon_valid  = lon[lony_valid-1]

# numbers of assimilated pixels
num_assim_path = pm.patch_filedir()+'/obs_patch/countnum.bin'
assim_pixel   = read_bin_int(num_assim_path,2) 
assim_pixel_num = assim_pixel[0,:,:]

assim_num = np.zeros(np.shape(lony_valid)[0])
for i in range(0,np.shape(lony_valid)[0]):
    assim_num[i] = assim_pixel_num[int(latx_valid[i]-1),int(lony_valid[i]-1)]

lat_valid_plot = np.delete(latx_valid,assim_num==0)
lon_valid_plot = np.delete(lony_valid,assim_num==0)
assim_num_plot = np.delete(assim_num,assim_num==0)

coords = np.stack((lat_valid_plot,lon_valid_plot),axis=1)
_,unique_indices = np.unique(coords,axis=0,return_index = True)  # delete repeated elements
unique_ind_sorted = np.sort(unique_indices)
lat_unique = lat_valid_plot[unique_ind_sorted]
lon_unique = lon_valid_plot[unique_ind_sorted]
assim_unique = assim_num_plot[unique_ind_sorted]
sorted_indices = np.lexsort((lon_unique, lat_unique))
lat_valid_plot = lat_unique[sorted_indices]
lon_valid_plot = lon_unique[sorted_indices]
assim_num_plot = assim_unique[sorted_indices]

#%% draw the select stations
def draw_select_station(lon_valid,lat_valid,assim_num,title):
    fig = plt.figure(dpi = 600,figsize=(7.5,4.5))
    ax1 = fig.add_axes([0.1,0.1,0.8,0.8],projection=ccrs.PlateCarree())
    ax1.set_axis_off()
    # ax1.outline_patch.set_linewidth(0.35)
    ax1.set_extent([128,149,30,46], ccrs.PlateCarree())
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=0, linestyle='-.')
    ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
    
    # mask the other place
    region_lon1 = [128.06, 128.06,138, 138 ]
    region_lat1 = [41 , 45.95 , 45.95, 41  ]
    region_lon2 = [128.06,128.06,131.4,131.4 ]
    region_lat2 = [34.5,41,41,34.5]
    ax1.fill(region_lon1, region_lat1, color='white', transform=ccrs.PlateCarree(), zorder=10)
    ax1.fill(region_lon2, region_lat2, color='white', transform=ccrs.PlateCarree(), zorder=10)
    
    gl.top_labels   = False
    gl.right_labels = False
    gl.bottom_labels   = False
    gl.left_labels = False
    # con_plot1 = ax1.scatter(lon[ix],lat[iy],s=1,zorder=2,color='black',transform=ccrs.PlateCarree(),alpha=0.95)
    con_plot1 = ax1.scatter(lon_select,lat_select,s=1,zorder=2,color='gray',transform=ccrs.PlateCarree(),alpha=0.95)
    con_plot2 = ax1.scatter(lon_valid,lat_valid,s=assim_num*2,zorder=3,edgecolor='red',facecolors='none',lw=1.,transform=ccrs.PlateCarree())
    con_plot2 = ax1.scatter(lon_valid,lat_valid,s=assim_num*2,zorder=3,edgecolor='red',facecolors='none',lw=1.,transform=ccrs.PlateCarree())
    con_plot2 = ax1.scatter(137.10,34.89,s=20,zorder=3,edgecolor='red',facecolors='none',lw=1.,transform=ccrs.PlateCarree())
    def color_bar(l,b,w,h):
      rect = [l,b,w,h]
      return cbar_ax
    [l1,b1,w1,h1] = [0.63,0.12,0.01,0.25]
    [l2,b2,w2,h2] = [0.63,0.32,0.01,0.25]
    # cbar_ax2 = color_bar(l2,b2,w2,h2)
    cmap_blue = ListedColormap(['skyblue']) 
    ax1.pcolormesh(lon2d_low_map,lat2d_low_map,up_low,cmap=cmap_blue,zorder=1, transform=ccrs.PlateCarree(),alpha=1.)  

    # cb1.ax.set_title('NRMSE \n $(m^{3}/s)$',fontdict=font_label)
    # cb2 = plt.colorbar(con_plot2, cax=cbar_ax2,orientation="vertical",shrink=0.5)
    # ax1.set_title(title,loc='center')
    if 'round' in pm_path:
        plt.savefig(inputdir+title+pm_path[-4]+'.jpg', format='jpg',dpi=600)
    else:
        plt.savefig(inputdir+title+'.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
draw_select_station(lon_valid,lat_valid,assim_num,'/1.stations')

In [ ]:
#%% interactive map
def plot_interactive():
    situ_select_loc = np.zeros((np.shape(lat_select)[0],2))
    situ_valid_loc  = np.zeros((np.shape(lat_valid)[0],2))
    # lat,lon
    situ_select_loc[:,0] = lat_select
    situ_select_loc[:,1] = lon_select
    situ_valid_loc[:,0] = lat_valid
    situ_valid_loc[:,1] = lon_valid
    lat_cen = 40
    lon_cen = 141
    my_map = folium.Map(location=(lat_cen,lon_cen),width=1000,height=600, zoom_start=10,attr="Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.")
    
    # Save the map to an HTML file
    ind = 0
    for coord in situ_select_loc:
        # read related data
        folium.CircleMarker(
            location=coord,
            radius=1,
            color='black',  # Assign color based on index
            fill=True,
            fill_color='black',
            fill_opacity=0.6,
        ).add_to(my_map)
        ind = ind+1
    for coord in situ_valid_loc:
        # read related data
        folium.CircleMarker(
            location=coord,
            radius=5,
            color='red',  # Assign color based on index
            fill=True,
            fill_color='red',
            fill_opacity=0.8,
        ).add_to(my_map)
        ind = ind+1
    if 'round' in pm_path:
        my_map.save('valid_map'+pm_path[-4]+'.html')
    else:
        my_map.save('valid_map.html')
    return my_map

my_map = plot_interactive()
my_map

In [ ]:
# read DA output
riv_outdir = inputdir + '/exp_' + expname + dahour_str + '/rivdph/'
dis_outdir = inputdir + '/exp_' + expname + dahour_str + '/outflw/'
new_row = ['da'+dahour_str]

def read_obs(var):
    current_time = stime
    data_obs = np.full((time_len,len(lon_valid_plot)),np.nan)
    ind = 0
    while current_time<etime:
        ctime = current_time.strftime('%Y%m%d%H')
        if var=='wlv':
            fname = obsdir+'/'+ctime+'.bin'
        else:
            fname = '/data42/yingying/obs/2019UST/dis/'+ctime+'.bin'
        data_each = read_bin2d(fname)
        data_each = np.where(data_each<-900,np.nan,data_each)
        for sta_ind in range(0,len(lat_valid_plot)):
            data_obs[ind,sta_ind] = data_each[int(lat_valid_plot[sta_ind])-1,int(lon_valid_plot[sta_ind])-1]
        current_time += timedelta(hours=1)
        ind = ind + 1
    return data_obs
elvmean  = read_bin2d(obsdir+'/elv_mean2019.bin')
elvloc = np.full((len(lon_valid_plot)),np.nan)
for sta_ind in range(len(lon_valid_plot)):
    elvloc[sta_ind] = elvmean[int(lat_valid_plot[sta_ind])-1,int(lon_valid_plot[sta_ind])-1]
    
# move the files from pm.outdir() to ./yyyy/pm.outdir()
if (pm.val_mode())==0:
    outdir = pm.DA_dir()+'/out/'
else:
    outdir = pm.DA_dir()+'/'+str(syyyy)+'/out_valid_round'+str(pm.val_round(pm.val_mode()))
    
fmean = outdir + '/simmean/'+expname+'/C000.bin'
simmean  = read_bin2d(fmean)

# read rainfall
if 'long' in pm.__file__:
    prep_path = '/data42/yingying/data/matsiro_frc/'
    # rainf = np.full((time_len,len(lon_valid_plot)),np.nan)
    # ind = 0
    # for mon in range(0,12):
    #     mon_str  = '{:02d}'.format(mon+1)
    #     lat,lon,rainf_mon = read_prep(prep_path+'tej_rain'+mon_str+'.nc')
    #     if mon == 0:
    #         # startdate start from 20240120
    #         diff = stime-datetime(2024,1,1,0)
    #         diff_hours = int(diff.total_seconds() / 3600)
    #         rainf_mon  = rainf_mon[diff_hours:,:,:]
    #     if mon == 11:
    #         # enddate is 20241230
    #         diff = datetime(2025,1,1,0)-etime
    #         diff_hours = diff.total_seconds() / 3600
    #         end_hour   = -int(np.shape(rainf_mon)[0]-diff_hours)-1
    #         print('end_hour:',end_hour)
    #         rainf_mon  = rainf_mon[:end_hour,:,:] 
    #     number = np.shape(rainf_mon)[0]
    #     for sta_ind in range(len(lon_valid_plot)):
    #         rainf[ind:int(ind+number),sta_ind] = rainf_mon[:,int(lat_valid_plot[sta_ind])-1,int(lon_valid_plot[sta_ind])-1]
    #     ind = ind + number
    rainf = np.load(prep_path+'rain2024.npy')
else:
    rainf = rainf[6*24:16*24,:,:]

def read_stations(var):
    if var == 'rivdph':
        if 'long' in pm.__file__:
            # load data to save time
            data_obs_wlv = np.load('/data42/yingying/obs/2024UST/wlv/2024wlv.npy')
        else:
            data_obs_wlv = read_obs('wlv')
        if 'round' in pm_path:
            filepath = riv_outdir + 'valid/round'+pm_path[-4]+'/'
        elif 'long' in pm_path:
            filepath = riv_outdir + 'long/'
        else:
            filepath = riv_outdir + 'valid/'
        data_obs = data_obs_wlv - elvloc
    else:
        data_obs = read_obs('dis')
        # if 'long' in pm.__file__:
        # np.save('/data42/yingying/obs/2024UST/dis/2024dis.npy',data_obs)
        if 'round' in pm_path:
            filepath = dis_outdir + 'valid/round'+pm_path[-4]+'/'
        elif 'long' in pm_path:
            filepath = dis_outdir + 'long/'
        else:
            filepath = dis_outdir + 'valid/'
    da_station = np.full((time_len,pm.ens_mem(),len(lon_valid_plot)),np.nan)
    sim_station = np.full((time_len,pm.ens_mem(),len(lon_valid_plot)),np.nan)
    mean_station = np.full((len(lon_valid_plot)),np.nan)
    obs_station  = data_obs
    if 'long' in pm.__file__:
        prep_station = rainf
    else:
        prep_station = np.full((time_len,len(lon_valid_plot)),np.nan)
    for sta_ind in range(len(lon_valid_plot)):
        if 'long' not in pm.__file__:
            prep_station[:,sta_ind] = rainf[:,int(lat_valid_plot[sta_ind])-1,int(lon_valid_plot[sta_ind])-1]
        mean_station[sta_ind] = simmean[int(lat_valid_plot[sta_ind])-1,int(lon_valid_plot[sta_ind])-1]
        for ens in range(0,pm.ens_mem()):
            fname = '{:04d}'.format(int(lat_valid_plot[sta_ind])) +  '{:04d}'.format(int(lon_valid_plot[sta_ind])) + '{:02d}'.format(ens+1)
            da_station[:,ens,sta_ind] = read_bin_station(filepath+fname + 'A.bin')
            sim_station[:,ens,sta_ind] = read_bin_station(filepath+fname + 'C.bin')
    return data_obs,da_station,sim_station,obs_station,mean_station,prep_station
data_obs_wlv,da_station,sim_station,obs_station,mean_station,prep_station = read_stations('rivdph')
data_obs_dis,da_station_dis,sim_station_dis,obs_station_dis,mean_station_dis,prep_station_dis = read_stations('outflw')

In [ ]:
# standard simulation is ens20 [-1]
# plot station wlv validation
def cal_rmse(y_true,y_pred,varname):
    len_obs = np.shape(np.where(~np.isnan(y_true)))[1]
    if "Outflow" in varname:
        if np.shape(np.where(~np.isnan(y_true[24*2:24*4])))[1]<10:
            rmse = np.nan
        else:
            rmse = np.sqrt(np.nanmean((y_pred-y_true)**2))
    if "WSE" in varname:
        if len_obs<int(time_len)/4:
            rmse = np.nan
        else:
            rmse = np.sqrt(np.nanmean((y_pred-y_true)**2))
    return rmse

def draw_lines(varname,obs_station,sim_station,da_station,savename,new_row):
    rows_old, cols = 7, 5
    fig,axes = plt.subplots(rows_old, cols, dpi = 300,figsize=(25,15))
    fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.25)
    time    = np.arange(0,time_len,1)
    valid_indices = [i for i in range(len(lon_valid_plot)) if not np.all(np.isnan(obs_station[tstart:, i]))]
    if "Outflow" in varname:
        # to remove the very small river stations
        valid_indices = [i for i in range(len(lon_valid_plot)) if not (np.all(np.isnan(obs_station[tstart:, i])) or np.all(obs_station[tstart:, i] < 5))]
    improve_rate = np.zeros(len(valid_indices))
    num_valid = len(valid_indices)
    rows = (num_valid + cols - 1) // cols
    # plot stations
    plot_idx = 0
    lon_point = []
    lat_point = []
    lon_point_ind = []
    lat_point_ind = []
    obs_point_ind = []
    sim_point_ind = []
    da_point_ind = []
    num_point = []
    for ind, i in enumerate(valid_indices):
        time_plot= time[tstart:]
        obs_plot = obs_station[tstart:,i]
        if "WSE" in varname:
            sim_plot = sim_station[tstart:,:,i]-mean_station[i]
            da_plot  = da_station[tstart:,:,i]-mean_station[i]
        else:
            sim_plot = sim_station[tstart:,:,i]
            da_plot  = da_station[tstart:,:,i]
        # RMSE
        rmse_da = cal_rmse(obs_plot,np.nanmean(da_plot,axis=1),varname)
        rmse_da_str = '{:.2f}'.format(rmse_da)
        rmse_sim= cal_rmse(obs_plot,np.nanmean(sim_plot,axis=1),varname)  # compare with the average
        # rmse_sim= cal_rmse(obs_plot,sim_plot[:,-1],varname)  # compare with open loop
        rmse_sim_str= '{:.2f}'.format(rmse_sim)
        if (np.isnan(rmse_da)) | (np.isnan(rmse_sim)):
            improve_rate[ind] = np.nan
            continue
        if (np.shape(np.unique(da_plot))[0]<2):  # exclude the fail DA result
            improve_rate[ind] = np.nan
            continue
        if (np.abs(np.nanmax(obs_plot))<0.1):  # exclude the gauges at small rivers
            improve_rate[ind] = np.nan
            continue            
        # Rainfall
        row, col = divmod(plot_idx,cols)
        ax2 = axes[row,col].twinx()
        ax2.invert_yaxis()
        ax2.bar(time[tstart:],prep_station[tstart:,i],color='gray',alpha=0.8)
        ax2.set_ylim(np.nanmax(prep_station[tstart:,i])+5,0)
        ax2.spines['right'].set_color('gray')
        ax2.yaxis.label.set_color('gray')
        ax2.tick_params(axis='y',colors='gray') 
        # simulations
        sim_point_ind = sim_point_ind + [sim_plot]
        axes[row,col].plot(time_plot,sim_plot,'b-',linewidth=0.15,alpha=0.8)
        # online DA
        da_point_ind  = da_point_ind + [da_plot]
        axes[row,col].plot(time_plot,da_plot,'r-',linewidth=1.25)
        # print(np.unique(da_plot))
        axes[row,col].set_xticks(np.arange(tstart+15,time_len,24))
        axes[row,col].set_xlim(tstart,time_len)
        # open loop
        # axes[row,col].plot(time_plot,sim_plot[:,-1],'b-',linewidth=1.5)
        axes[row,col].plot(time_plot,np.nanmean(sim_plot,axis=1),'b-',linewidth=1.5)
        # obs
        obs_point_ind = obs_point_ind + [obs_plot]
        axes[row,col].plot(time_plot,obs_plot,color='green',linewidth=2.5)
        lat_str = '{:.2f}'.format(lat[int(lat_valid_plot[i])-1])
        # lat_str = '{:.2f}'.format(int(lat_valid_plot[i])-1)
        lon_str = '{:.2f}'.format(lon[int(lon_valid_plot[i])-1])
        # lon_str = '{:.2f}'.format(int(lon_valid_plot[i])-1)
        if rmse_da < rmse_sim:
            improve_rate[ind] = 1
        axes[row,col].text(0.02,0.95,'('+lat_str+','+lon_str+')\n' + 'obs number: '+str(int(assim_num_plot[i])),transform=axes[row,col].transAxes,fontsize=8, c='black',fontweight='bold', va='top', ha='left')
        axes[row,col].text(0.9,0.95,'RMSE: '+rmse_da_str,transform=axes[row,col].transAxes,fontsize=8, c='red',fontweight='bold', va='top', ha='right')
        axes[row,col].text(0.9,0.85,'RMSE: '+rmse_sim_str,transform=axes[row,col].transAxes,fontsize=8, c='blue',fontweight='bold', va='top', ha='right')
        if col == 0:
            axes[row,col].set_ylabel(varname)
        if col == cols-1:
            ax2.set_ylabel('Rain (mm/h)')
        if 'ils01' in pm_path or 'round' or 'msm' in pm_path:
            if 'WSE' in varname:
                if row == 5: # parmas01
                    dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
                    axes[row,col].set_xticklabels(dates,rotation=25)
                else:
                    axes[row,col].set_xticklabels([],rotation=15)
            else:
                    if row == rows-1:
                        dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
                        axes[row,col].set_xticklabels(dates,rotation=25)
                    else:
                        axes[row,col].set_xticklabels([],rotation=15)     
        if 'ils03' in pm_path:
            if 'WSE' in varname:
                if row == 5: # parmas03
                    dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
                    axes[row,col].set_xticklabels(dates,rotation=25)
                else:
                    axes[row,col].set_xticklabels([],rotation=15)
            else:
                    if row == 2:
                        dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
                        axes[row,col].set_xticklabels(dates,rotation=25)
                    else:
                        axes[row,col].set_xticklabels([],rotation=15) 
        plot_idx = plot_idx + 1
        lon_point = lon_point + [lon[int(lon_valid_plot[i])-1]]
        lat_point = lat_point + [lat[int(lat_valid_plot[i])-1]]
        lon_point_ind  = lon_point_ind + [int(lon_valid_plot[i])-1]
        lat_point_ind  = lat_point_ind + [int(lat_valid_plot[i])-1]
        num_point = num_point + [assim_num_plot[i]]
    for ind in range(plot_idx, rows_old * cols):
        row, col = divmod(ind, cols)
        fig.delaxes(axes[row, col])  # delete the empty 
    improve = np.nansum(improve_rate)/np.shape(np.where(~np.isnan(improve_rate)))[1]*100
    print(np.nansum(improve_rate),np.shape(np.where(~np.isnan(improve_rate)))[1])
    print('Overall number of stations:',np.shape(np.where(assim_num!=0))[1])
    print('How many stations improve: ','{:.2f}'.format(improve),'%')
    new_row = new_row + ['{:.2f}'.format(improve)]
    if 'round' in pm_path:
        plt.savefig(exp_plotdir+'2.cross_validation'+savename+pm_path[-4]+'.jpg', format='jpg',dpi=600)
    elif 'long' in pm_path:
        plt.savefig(exp_plotdir+'2.cross_validation'+savename+'999.jpg', format='jpg',dpi=600)
    else:
        plt.savefig(exp_plotdir+'2.cross_validation'+savename+'.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
    return np.array(lon_point), np.array(lat_point), np.array(lon_point_ind), np.array(lat_point_ind), np.array(obs_point_ind), np.array(sim_point_ind), np.array(da_point_ind), np.array(num_point), new_row
lon_point, lat_point, lon_point_ind, lat_point_ind, obs_point_ind, sim_point_ind, da_point_ind, num_point, new_row = draw_lines('Anomaly WSE (m)',obs_station,sim_station,da_station,'_wlv',new_row)

# np.save(pm.plot_dir()+'/exp_'+expname+dahour_str+'/lon_point_ind.npy',lon_point_ind)
# np.save(pm.plot_dir()+'/exp_'+expname+dahour_str+'/lat_point_ind.npy',lat_point_ind)
# plot map
draw_select_station(lon_point, lat_point, num_point,'/1.plot_stations')

In [ ]:
# print improvement of rolling selection experiment
print('Average improvement of rolling result',np.nanmean([22/24,11/12,14/18,9/12,13/15]))

In [ ]:
# relationship between obs number and SVD, RMSE

In [ ]:
#%% outflow validation
lon_dis_point, lat_dis_point, lon_ind_test, lat_ind_test, obs_point_dis, sim_point_dis, da_point_dis, num_dis_point, new_row = draw_lines('Outflow (m${^3}$/s)',obs_station_dis,sim_station_dis,da_station_dis,'_HQ',new_row)
draw_select_station(lon_dis_point, lat_dis_point, num_dis_point,'/1.plot_dis_stations')
# draw_lines('Outflow (m${^3}$/s)',obs_station_dis,sim_station_dis,da_HQ,'_dis',new_row)

In [ ]:
#%% NRMSE
def draw_each_figure(obs_lat,obs_lon,varname):
    situ_rmse  = np.full(np.shape(obs_lat)[0],np.nan)
    situ_nrmse  = np.full(np.shape(obs_lat)[0],np.nan)
    lat_plot   = np.full(np.shape(obs_lat)[0],1)
    lon_plot   = np.full(np.shape(obs_lat)[0],1)
    situ_imp   = np.full(np.shape(obs_lat)[0],np.nan)
    peak_time_obs = np.full(np.shape(obs_lat)[0],np.nan)
    loc_row = np.full(np.shape(obs_lat)[0],np.nan)
    if 'round' in pm_path:
        filepath = inputdir+'/exp_'+expname+dahour_str+'/round'+pm_path[-4]+'/'+varname+'/'
    else:
        filepath = inputdir+'/exp_'+expname+dahour_str+'/'+varname+'/'
    # DA
    filename = filepath+'dataA_mean01_20.nc'   # compare to the average of ensemble
    da_mean = read_nc(filename,3)  # compare with the open loop result
    # sim
    filename = filepath+'dataC_mean01_20.nc'
    # filename = filepath+'var00.nc'
    if '00' in filename:
        sim_mean = read_nc(filename,4)
    else:
        sim_mean = read_nc(filename,3)
    for loc in range(0,np.shape(obs_lat)[0]):
        loc_lat = obs_lat[loc]
        loc_lon = obs_lon[loc]
        loc_id  = id_info[loc]
        #obs data_obs_wlv/data_obs_dis
        if varname == 'rivdph':
            obs_grid = data_obs_wlv[:,loc_lat,loc_lon]
        else:
            obs_grid = data_obs_dis[:,loc_lat,loc_lon]
        if np.all(obs_grid)<-900:
            continue
        else:
        # read data
            if varname == 'rivdph':
                loc_obs = obs_grid[tstart:]-elvmean[loc_lat,loc_lon]
                loc_sim = sim_mean[tstart:,int(loc_lat),int(loc_lon)]-simmean[loc_lat,loc_lon]
                loc_da  = da_mean[tstart:,loc_lat,loc_lon]-simmean[loc_lat,loc_lon]
            else:
                loc_obs = obs_grid[tstart:]
                loc_sim = sim_mean[tstart:,loc_lat,loc_lon]
                loc_da  = da_mean[tstart:,loc_lat,loc_lon]
            if np.all(loc_da[10:30]<10**(-8))==True:
                loc_da = np.full(time_len-tstart,np.nan)
            # remove no obs stations   
            mse_sim = np.nanmean((loc_obs-loc_sim)**2)
            mse_da  = np.nanmean((loc_obs-loc_da)**2)
            if np.isnan(mse_sim) | np.isnan(mse_da) | np.all(np.isnan(loc_da)):
                continue
            rmse = np.sqrt(mse_sim) - np.sqrt(mse_da)
            if (np.shape(loc_obs)[0]<5) | (np.shape(loc_sim)[0]<5) | (np.shape(loc_da)[0]<5):
                rmse = np.nan
                nrmse = np.nan
                imp = np.nan
            else:
                var_max  = np.nanmax(loc_obs)
                var_min  = np.nanmin(loc_obs)
                var_range = var_max-var_min
                if var_range ==0:
                    nrmse = np.nan
                else:
                    nrmse = rmse/var_range
                peak_time_obs_each = np.nanargmax(loc_obs)
                # if nrmse<-0.2:
                #     print(nrmse,rmse,loc_lat,loc_lon)
                imp = rmse/np.sqrt(mse_sim)            
            # store nrmse, lat, loc
            situ_rmse[loc]=rmse
            situ_nrmse[loc]=nrmse
            situ_imp[loc]=imp
            lat_plot[loc]=int(loc_lat)
            lon_plot[loc]=int(loc_lon)
            loc_row[loc]=loc
            peak_time_obs[loc]=peak_time_obs_each
    return situ_rmse,situ_nrmse,situ_imp,peak_time_obs,loc_row,lat_plot,lon_plot
# compare with rivdph
situ_rmse_wlv,situ_nrmse_wlv,situ_imp_wlv,peak_time_obs_wlv,loc_row_wlv,lat_plot_wlv,lon_plot_wlv = draw_each_figure(iy,ix,'rivdph')
# compare with outflow
situ_rmse_dis,situ_nrmse_dis,situ_imp_dis,peak_time_obs_dis,loc_row_dis,lat_plot_dis,lon_plot_dis = draw_each_figure(iy,ix,'outflw')

def performance(situ_rmse,situ_nrmse,situ_imp,new_row):
    if (np.shape(np.where(situ_nrmse>0))[1]+np.shape(np.where(situ_nrmse<0))[1]) == 0:
        nrmse_pos = np.nan
    else:
        nrmse_pos = np.shape(np.where(situ_nrmse>0))[1]/(np.shape(np.where(situ_nrmse>0))[1]+np.shape(np.where(situ_nrmse<0))[1])
    nrmse_neg = 1-nrmse_pos
    print('better',nrmse_pos*100,'worse',nrmse_neg*100)
    print('average performance',np.nanmean(situ_rmse),np.nanmean(situ_nrmse),np.nanmean(situ_imp)*100)
    values = [nrmse_pos*100, nrmse_neg*100, np.nanmean(situ_rmse), np.nanmean(situ_nrmse), np.nanmean(situ_imp)*100]
    new_row += ['{:.2f}'.format(v) for v in values]
    return new_row
# rivdph
new_row = performance(situ_rmse_wlv,situ_nrmse_wlv,situ_imp_wlv,new_row)
# outflow
new_row = performance(situ_rmse_dis,situ_nrmse_dis,situ_imp_dis,new_row)

In [ ]:
# store the values
column = ["da_hours", "wlv (%)", "dis (%)", 
    "wlv_better (%)", "wlv_worse (%)", "wlv_rmse", 
    "wlv_nrmse", "wlv_imp (%)", 
    "dis_better (%)", "dis_worse (%)", "dis_rmse", 
    "dis_nrmse", "dis_imp (%)"]
# df = pd.DataFrame(columns=column)
# df.to_csv(pm.plot_dir()+'/result.csv',index=False)
# new_row_pd = pd.DataFrame([new_row],columns=df.columns)
# new_row_pd.to_csv(pm.plot_dir()+'/result.csv', mode="a", index=False, header=False)


In [ ]:
#%% draw the map
def draw_rmse(title,var_name,lon_plot,lat_plot,situ_rmse,vmin,vmax):
    fig = plt.figure(dpi = 600,figsize=(7.5,4.5))
    ax1 = fig.add_axes([0.1,0.1,0.8,0.8],projection=ccrs.PlateCarree())
    ax1.set_axis_off()
    # ax1.outline_patch.set_linewidth(0.35)
    ax1.set_extent([128,149,30,46], ccrs.PlateCarree())
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=0, linestyle='-.')
    ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
    
    # mask the other place
    region_lon1 = [128.06, 128.06,138, 138 ]
    region_lat1 = [41 , 45.95 , 45.95, 41  ]
    region_lon2 = [128.06,128.06,131.4,131.4 ]
    region_lat2 = [34.5,41,41,34.5]
    ax1.fill(region_lon1, region_lat1, color='white', transform=ccrs.PlateCarree(), zorder=10)
    ax1.fill(region_lon2, region_lat2, color='white', transform=ccrs.PlateCarree(), zorder=10)
    
    gl.top_labels   = False
    gl.right_labels = False
    gl.bottom_labels   = False
    gl.left_labels = False
    # gl.xlocator = mticker.FixedLocator([130,135,140,145])
    # gl.ylocator = mticker.FixedLocator([30,35,40,45])
    # gl.xlabel_style = {'size': 8,'color':"#3F3A3A",'weight':'normal'}
    # gl.ylabel_style = {'size': 8,'color':"#3F3A3A",'weight':'normal'}
    # lon_formatter = LongitudeFormatter(zero_direction_label=True)
    # lat_formatter = LatitudeFormatter()
    # ax1.xaxis.set_major_formatter(lon_formatter)
    # ax1.yaxis.set_major_formatter(lat_formatter)
    norm = mcolors.TwoSlopeNorm(vmin=min(situ_rmse), vcenter=0, vmax=-min(situ_rmse))
    con_plot2 = ax1.scatter(lon[lon_plot],lat[lat_plot],c=situ_rmse,s=1.5,zorder=2,cmap='coolwarm',transform=ccrs.PlateCarree(),vmin=vmin,vmax=vmax,alpha=0.8)
    # station highlight  
    # ax1.scatter(lon[918],lat[556],c='black',alpha=0.5,zorder=3,transform=ccrs.PlateCarree(),marker='*',s=200)
    def color_bar(l,b,w,h):
      rect = [l,b,w,h]
      cbar_ax = fig.add_axes(rect)
      return cbar_ax
    [l1,b1,w1,h1] = [0.63,0.12,0.01,0.25]
    [l2,b2,w2,h2] = [0.63,0.32,0.01,0.25]
    cbar_ax2 = color_bar(l2,b2,w2,h2)
    if var_name == 'rivdph':
        cb_title = '$(m)$'
    else:
        cb_title = '$(m^{3}/s)$'
    cb2 = plt.colorbar(con_plot2, cax=cbar_ax2,orientation="vertical",shrink=0.5)
    cb2.ax.set_title(cb_title,fontsize=8)
    ax1.set_title(title,loc='center')
    if 'round' in pm_path:
        plt.savefig(exp_plotdir+'3.map_'+title+pm_path[-4]+'.jpg', format='jpg',dpi=600)
    elif 'long' in pm_path:
        plt.savefig(exp_plotdir+'3.map_'+title+pm_path[-4]+'.jpg', format='jpg',dpi=600)
        
    else:
        plt.savefig(exp_plotdir+'3.map_'+title+'.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
# wlv
draw_rmse('Rivdph_RMSE','rivdph',lon_plot_wlv,lat_plot_wlv,situ_rmse_wlv,-1,1)   
draw_rmse('Rivdph_nRMSE','rivdph',lon_plot_wlv,lat_plot_wlv,situ_nrmse_wlv,-0.8,0.8)   
draw_rmse('Rivdph_Improvement','rivdph',lon_plot_wlv,lat_plot_wlv,situ_imp_wlv,0,1)  
print('-------------------')
# dis
draw_rmse('Outflow_RMSE','outflow',lon_plot_dis,lat_plot_dis,situ_rmse_dis,-np.nanmax(situ_rmse_dis)*0.6,np.nanmax(situ_rmse_dis)*0.6) 
draw_rmse('Outflow_nRMSE','outflow',lon_plot_dis,lat_plot_dis,situ_nrmse_dis,-0.4,0.4)   
draw_rmse('Outflow_Improvement','outflow',lon_plot_dis,lat_plot_dis,situ_imp_dis,0,1)


In [ ]:
#%% draw observed-outflow
def cal_HQ():
    cal = 0
    Q_obs = np.full((np.shape(da_station_dis)[0],np.shape(da_station_dis)[1],np.shape(da_station_dis)[2]),np.nan)
    for station_ind in range(0,np.shape(obs_station)[1]):
        if np.shape(np.where(~np.isnan(obs_station_dis[tstart:,station_ind])))[1]<time_len/4:
            continue
        obs_wlv = obs_station[tstart:,station_ind]
        obs_dis = obs_station_dis[tstart:,station_ind]
        mask = ~np.isnan(obs_wlv) & ~np.isnan(obs_dis)
        obs_wlv_new = obs_wlv[mask]
        obs_dis_new = obs_dis[mask]
        coeffi = np.polyfit(obs_wlv_new,obs_dis_new,2) # second-order polynomic
        a, b, c = coeffi
        Q_fit  = a*obs_wlv_new**2 + b*obs_wlv_new + c
        R,_ = pearsonr(obs_dis_new,Q_fit)
        for ens_ind in range(0,np.shape(da_station)[1]):
            da_wlv  = da_station[tstart:,ens_ind,station_ind]
            Q_obs[tstart:,ens_ind,station_ind]  = a*da_wlv**2 + b*da_wlv + c
        if R<0.85:
            continue
        cal = cal + 1
    return Q_obs

da_station_disnew = cal_HQ()
# plot the fitting H-Q figure
lon_test, lat_test, lon_ind_test, lat_ind_test, obs_point_test, sim_point_test, da_point_test, num_test, new_test = draw_lines('Outflow (m${^3}$/s)',obs_station_dis,sim_station_dis,da_station_disnew,'_dis',new_row)

In [ ]:
# read 24h data
# make by code /data42/yingying/HydroDA/src/plot/7.make_24forecast.py
# [ens,time_len-24,stations,24hours]
wlv_da  = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/rivdph/da_24fore.npy')
wlv_sim = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/rivdph/sim_24fore.npy')
dis_da  = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/outflw/da_24fore.npy')
dis_sim = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/outflw/sim_24fore.npy')
lon_point_ind = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/lon_point_ind.npy')
lat_point_ind = np.load(pm.plot_dir()+'/exp_'+expname+dahour_str+'/lat_point_ind.npy')
# original runoff forcing: dis_da_all[-1,:,:,:]
dis_da_all = np.full((20, int(216/dahour), np.shape(lon_point_ind)[0], 24),np.nan)
dis_da_all[:,int(24*2/dahour):,:,:] = dis_da
wlv_da_all = np.full((20, int(216/dahour), np.shape(lon_point_ind)[0], 24),np.nan)
wlv_da_all[:,int(24*2/dahour):,:,:] = wlv_da
wlv_da = wlv_da_all

In [ ]:
# plot station 24 hour forecast wlv validation
def plot_gradient_line(ax, x, y, cmap, linewidth=1.0, alpha=0.8, vmin=None, vmax=None,add_colorbar=False,cbar_label='leading time',fig=None):
    x = np.asarray(x)
    y = np.asarray(y)
    if len(x) != len(y):
        raise ValueError("x and y must be the same length")
    if len(x) < 2:
        return  # Don't do anything
    if isinstance(cmap, str):
        cmap = plt.colormaps.get_cmap(cmap)
        colors = cmap(np.linspace(0.15, 0.85, len(x)-1))
    if vmin is None:
        vmin = 0
    if vmax is None:
        vmax = len(x) - 2
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = LineCollection(segments, cmap=cmap, norm=norm,
                        linewidth=linewidth, alpha=alpha)
    lc.set_array(np.arange(len(x)-1)) 
    ax.add_collection(lc)
    if add_colorbar and fig is not None:
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array([])  # no need for the real value
        ticks = np.linspace(0,len(x)-1,len(x)).astype(int)+1
        cbar = fig.colorbar(sm, ax=ax, pad=0.01)
        cbar.set_label(cbar_label) 
        cbar.ax.tick_params(labelsize=15)  
        cbar.ax.yaxis.set_ticks_position('right') 
        cbar.set_ticks(ticks-1)
        cbar.set_ticklabels(ticks)
        cbar.ax.set_position([0.92, 0.5, 0.02, 0.4]) 
    return lc 

def draw_fore_lines(varname,obs_station,sim_station,da_station,wlv_da,savename,station_lat,station_lon,station_idx):
    rows_old, cols = 7, 5
    fig,axes = plt.subplots(rows_old, cols, dpi = 300,figsize=(25,15))
    fig.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.25)
    time    = np.arange(0,time_len,1)
    valid_indices = [i for i in range(len(lon_valid_plot)) if not np.all(np.isnan(obs_station[tstart:, i]))]
    if "Outflow" in varname:
        # to remove the very small river stations
        valid_indices = [i for i in range(len(lon_valid_plot)) if not (np.all(np.isnan(obs_station[tstart:, i])) or np.all(obs_station[tstart:, i] < 5))]
    improve_rate = np.zeros(len(valid_indices))
    num_valid = len(valid_indices)
    rows = (num_valid + cols - 1) // cols
    # plot stations
    plot_idx = 0
    dis_use_ind = 0
    lon_point = []
    lat_point = []
    lon_point_ind = []
    lat_point_ind = []
    num_point = []
    dis_da_use = np.full((np.shape(wlv_da)[0],np.shape(wlv_da)[1],np.shape(obs_point_dis)[0],np.shape(wlv_da)[3]),np.nan)
    for ind, i in enumerate(valid_indices):
        time_plot= time[tstart:]
        obs_plot = obs_station[tstart:,i]
        if "WSE" in varname:
            sim_plot = sim_station[tstart:,:,i]-mean_station[i]
            da_plot  = da_station[tstart:,:,i]-mean_station[i]
        else:
            sim_plot = sim_station[tstart:,:,i]
            da_plot  = da_station[tstart:,:,i]
        # RMSE (compare the original data)
        # da_plot_singleline  = da_plot[:,-1]
        # sim_plot_singleline = sim_plot[:,-1]
        da_plot_singleline  = np.nanmean(da_plot,axis=1)
        sim_plot_singleline = np.nanmean(sim_plot,axis=1)
        rmse_da = cal_rmse(obs_plot,da_plot_singleline,varname)
        rmse_da_str = '{:.2f}'.format(rmse_da)
        rmse_sim= cal_rmse(obs_plot,sim_plot_singleline,varname)
        rmse_sim_str= '{:.2f}'.format(rmse_sim)
        if (np.isnan(rmse_da)) | (np.isnan(rmse_sim)):
            improve_rate[ind] = np.nan
            continue
        if (np.shape(np.unique(da_plot))[0]<2):  # exclude the fail DA result
            improve_rate[ind] = np.nan
            continue
        if (np.abs(np.nanmax(obs_plot))<0.1):  # exclude the gauges at small rivers
            improve_rate[ind] = np.nan
            continue            
        # Rainfall
        row, col = divmod(plot_idx,cols)
        ax2 = axes[row,col].twinx()
        ax2.invert_yaxis()
        ax2.bar(time[tstart:],prep_station[tstart:,i],color='gray',alpha=0.8)
        ax2.set_ylim(np.nanmax(prep_station[tstart:,i])+5,0)
        ax2.spines['right'].set_color('gray')
        ax2.yaxis.label.set_color('gray')
        ax2.tick_params(axis='y',colors='gray') 
        # obs
        axes[row,col].plot(time_plot,obs_plot,color='green',linewidth=2.)
        axes[row,col].set_xticks(np.arange(tstart+15,time_len,24))
        axes[row,col].set_xlim(tstart,time_len)
        lat_str = '{:.2f}'.format(lat[int(lat_valid_plot[i])-1])
        # lat_str = '{:.2f}'.format(int(lat_valid_plot[i])-1)
        lon_str = '{:.2f}'.format(lon[int(lon_valid_plot[i])-1])
        # lon_str = '{:.2f}'.format(int(lon_valid_plot[i])-1)
        # use redish color to plot lines
        # simulations
        axes[row,col].plot(time_plot,sim_plot_singleline,'b-',linewidth=1.5)
        if "WSE" in varname:
            station_lat = station_lat + [int(lat_valid_plot[i])-1]
            station_lon = station_lon + [int(lon_valid_plot[i])-1]
            station_idx  = station_idx  + [plot_idx]
        for time_ind in range(tstart_da,np.shape(wlv_da)[1],1):
            # for ens_ind in range(0,pm.ens_mem()):
            #     # online DA
            #     axes[row,col].plot(np.arange(time_ind,time_ind+24,1),wlv_sim[ens_ind,time_ind,plot_idx,:],'b-',linewidth=0.25)
            x_time = np.arange(time_ind*dahour,time_ind*dahour+24,1)
            if "Outflow" in varname:
                station_ind = np.where((station_lat==int(lat_valid_plot[i]-1)) & (station_lon==int(lon_valid_plot[i]-1)))[0][0]
                y_value = np.nanmean(wlv_da[:,time_ind,int(station_idx[int(station_ind)]),:],axis=0)
                # only store the data for stations have water discharge observations
                dis_da_use[:,time_ind,dis_use_ind,:] =  wlv_da[:,time_ind,int(station_idx[int(station_ind)]),:]
            else:
                y_value = np.nanmean(wlv_da[:,time_ind,plot_idx,:],axis=0)
            plot_gradient_line(axes[row, col], x_time, y_value, 'Reds_r', linewidth=0.5,add_colorbar=(time_ind == tstart),fig=fig)        
        if rmse_da < rmse_sim:
            improve_rate[ind] = 1
        axes[row,col].text(0.02,0.95,'('+lat_str+','+lon_str+')\n' + 'obs number: '+str(int(assim_num_plot[i])),transform=axes[row,col].transAxes,fontsize=8, c='black',fontweight='bold', va='top', ha='left')
        if col == 0:
            axes[row,col].set_ylabel(varname)
        if col == cols-1:
            ax2.set_ylabel('Rain (mm/h)')
        if row != rows-1:
            axes[row,col].set_xticklabels([],rotation=15)
        if ("WSE" in varname) & (row == 5):
            dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
            axes[row,col].set_xticklabels(dates,rotation=25)
        if ("Outflow" in varname) & (row == 3):
            dates = pd.date_range(start='2019-10-11 00:00',end='2019-10-17 08:00',freq='24h')
            axes[row,col].set_xticklabels(dates,rotation=25)            
        plot_idx = plot_idx + 1
        lon_point = lon_point + [lon[int(lon_valid_plot[i])-1]]
        lat_point = lat_point + [lat[int(lat_valid_plot[i])-1]]
        lon_point_ind  = lon_point_ind + [int(lon_valid_plot[i])-1]
        lat_point_ind  = lat_point_ind + [int(lat_valid_plot[i])-1]
        num_point = num_point + [assim_num_plot[i]]
        dis_use_ind = dis_use_ind + 1
    for ind in range(plot_idx, rows_old * cols):
        row, col = divmod(ind, cols)
        fig.delaxes(axes[row, col])  # delete the empty 
    improve = np.nansum(improve_rate)/np.shape(np.where(~np.isnan(improve_rate)))[1]*100
    print('Overall number of stations:',np.shape(np.where(assim_num!=0))[1])
    print('How many stations improve: ','{:.2f}'.format(improve),'%')
    plt.savefig(exp_plotdir+'2.fore_validation'+savename+'.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
    if "WSE" in varname:
        return np.array(station_lat),np.array(station_lon),np.array(station_idx),None
    else:
        return None,None,None,dis_da_use
station_lat,station_lon,station_idx,wlv_da_test = draw_fore_lines('Anomaly WSE (m)',obs_station,sim_station,da_station,wlv_da,'_wlv',[],[],[])


In [ ]:
# dis_da_use: only store the water discharge at stations with observation (12 stations)
# wlv_da: WSE at stations with observations (24 stations)
lat_test,lon_test,idx_test,dis_da_use = draw_fore_lines('Outflow (m${^3}$/s)',obs_station_dis,sim_station_dis,da_station_dis,dis_da_all,'_dis',station_lat,station_lon,station_idx)

In [ ]:
print(np.shape(wlv_sim))

In [ ]:
# calculate KGE, Timing error, Person Correlation for various leading time
def cal_single_metric(obs_data,sim_data):
    if np.shape(obs_data)[0] != np.shape(sim_data)[0]:
        print('shape:',np.shape(obs_data)[0],np.shape(sim_data)[0])
    mask = ~np.isnan(obs_data) & ~np.isnan(sim_data)
    if np.shape(sim_data)[0]>20:
        obs = obs_data[mask]
        sim = sim_data[mask]
    else:
        obs = obs_data
        sim = sim_data
    if ((np.all(mask) == True) & (np.shape(sim)[0]>1)) | (np.shape(sim)[0]>20):
        rmse = np.sqrt(np.mean((sim-obs)**2))
        bias = np.mean(sim-obs)
        # kge
        r = np.corrcoef(sim, obs)[0, 1]
        alpha = np.std(sim) / np.std(obs)
        beta = np.mean(sim) / np.mean(obs)
        kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
        # r
        pearson_r, _ = pearsonr(obs, sim)
        # Timing error (lag of peak)
        obs_peak = np.argmax(obs)
        sim_peak = np.argmax(sim)
        time_error = sim_peak - obs_peak
        return rmse,bias,kge,time_error,pearson_r
    else:
        return np.nan,np.nan,np.nan,np.nan,np.nan

def evaluate_fore(wlv_sim):
    time_data = np.arange(0,23,dahour)
    rmse_compare = np.full((np.shape(obs_point_ind)[0],int((time_len-tstart-24)/dahour),len(time_data)),np.nan)
    bias_compare = np.full((np.shape(obs_point_ind)[0],int((time_len-tstart-24)/dahour),len(time_data)),np.nan)
    kge_compare = np.full((np.shape(obs_point_ind)[0],int((time_len-tstart-24)/dahour),len(time_data)),np.nan)
    time_error_compare = np.full((np.shape(obs_point_ind)[0],int((time_len-tstart-24)/dahour),len(time_data)),np.nan)
    pearson_r_compare = np.full((np.shape(obs_point_ind)[0],int((time_len-tstart-24)/dahour),len(time_data)),np.nan)
    test_time = 0
    for time_ind in range(0,time_len-tstart-24,dahour):
        for station_ind in range(0,np.shape(obs_point_ind)[0]):
            if time_ind>24-1:
                if 'ils01' in pm_path or 'round' in pm_path:
                    continue
                else:
                    sim_data = np.nanmean(wlv_sim[:,tstart_da+int((time_ind-24)/dahour)+1:tstart_da+int(time_ind/dahour)+1,station_ind,:],axis=0)
                    compare_lead_time = 23
            else:
                sim_data = np.nanmean(wlv_sim[:,tstart_da:tstart_da+int(time_ind/dahour)+1,station_ind,:],axis=0)
                compare_lead_time = time_ind
            for compare_ind in range(0,compare_lead_time,dahour):
                # at least 1 hour ahead
                if compare_lead_time-compare_ind+1>24:
                    obs_len  = 24-(compare_lead_time-compare_ind-dahour)
                    obs_data = obs_station[time_ind:time_ind+obs_len,station_ind]
                    sim_each = sim_data[int(compare_ind/dahour),compare_lead_time-compare_ind-dahour:24]
                else:
                    obs_data = obs_station[time_ind:time_ind+dahour+1,station_ind]
                    sim_each = sim_data[int(compare_ind/dahour),compare_lead_time-compare_ind-dahour:compare_lead_time-compare_ind+1]
                rmse_compare[station_ind,int(time_ind/dahour),int(compare_ind/dahour)],bias_compare[station_ind,int(time_ind/dahour),int(compare_ind/dahour)],kge_compare[station_ind,int(time_ind/dahour),int(compare_ind/dahour)],time_error_compare[station_ind,int(time_ind/dahour),int(compare_ind/dahour)],pearson_r_compare[station_ind,int(time_ind/dahour),int(compare_ind/dahour)] = cal_single_metric(obs_data,sim_each)
                test_time = test_time + 1
                if np.shape(sim_each)[0]<2:
                    print(time_ind,station_ind,compare_ind,compare_lead_time,obs_data,sim_each)
                    exit()
    # save leading time from 24 hours to 1 hour
    rmse = np.nanmean(rmse_compare, axis=(1))
    bias = np.nanmean(bias_compare, axis=(1))
    kge  = np.nanmean(kge_compare, axis=(1))
    time_error = np.nanmean(time_error_compare, axis=(1))
    pearson_r = np.nanmean(pearson_r_compare, axis=(1))
    # save leading time from 1 hour to 24 hours
    return rmse[:,::-1],bias[:,::-1],kge[:,::-1]
rmse_da,bias_da,kge_da = evaluate_fore(wlv_da)

def cal_metric_sim():
    metric_sim = np.full((np.shape(obs_point_ind)[0],3),np.nan)
    for station_ind in range(0,np.shape(obs_point_ind)[0]):
        obs_data = obs_point_ind[station_ind,:]
        sim_data = np.nanmean(sim_point_ind[station_ind,:,:],axis=1)
        metric_sim[station_ind,0],metric_sim[station_ind,1],metric_sim[station_ind,2],time_error,pearson_r = cal_single_metric(obs_data,sim_data)
    return metric_sim
# rmse,bias,kge
metric_sim = cal_metric_sim()


In [ ]:
# calculate time error
def evaluate_timeerror(wlv_da):
    time_date = np.arange(0,23,dahour)
    time_error_all = np.full((np.shape(obs_point_ind)[0],len(time_date)),np.nan) # from 1 to 24 hour leading time
    time_error_sim = np.full((np.shape(obs_point_ind)[0]),np.nan) # from 1 to 24 hour leading time
    for station_ind in range(0,np.shape(obs_point_ind)[0]):
        obs_data = obs_point_ind[station_ind,:]
        obs_peak = np.argmax(obs_data) + tstart
        sim_mean = np.nanmean(sim_point_ind,axis=2)
        sim_data = sim_mean[station_ind,:]
        sim_peak = np.argmax(sim_data) + tstart
        time_error_sim[station_ind]  = sim_peak-obs_peak
        obs_peak_ind = int((obs_peak-1)/dahour)
        obs_ind_remain = obs_peak-obs_peak_ind*dahour
        if obs_peak_ind-tstart_da>int(24/dahour):
            da_data_all = wlv_da[:,obs_peak_ind-int(24/dahour)+1:obs_peak_ind+1,:,:]
            da_data = np.nanmean(da_data_all[:,:,station_ind,:],axis=0)
            for time_ind in range(0,23,dahour):
                sim_peak = np.argmax(da_data[int(time_ind/dahour),:])
                # time_error_all[station_ind,int(time_ind/dahour)] = sim_peak-(23-time_ind) 
                time_error_all[station_ind,int(time_ind/dahour)] = sim_peak-(24-int(time_ind/dahour)*dahour-obs_ind_remain) 
                # <0 earlier then real
                # >0 later then real
    # [stations, leadtime]
    time_error_mean = np.nanmean(time_error_all,axis=0)
    return time_error_all[:,::-1],time_error_sim
# [station,24hour]
time_error_da,time_error_sim  = evaluate_timeerror(wlv_da)

In [ ]:
# plot bar error
def plot_bar_error(ax,data_sim,data_da,ymin,ymax,ylabel):
    def get_stats(data):
        median = np.nanpercentile(data, 50, axis=0)
        q25 = np.nanpercentile(data, 25, axis=0)
        q75 = np.nanpercentile(data, 75, axis=0)
        q10 = np.nanpercentile(data, 10, axis=0)
        q90 = np.nanpercentile(data, 90, axis=0)
        err_lower = median - q10
        err_upper = q90 - median
        return median,q25,q75,err_lower,err_upper
    # sim
    med1 = np.nanpercentile(data_sim, 50)
    q25_1 = np.nanpercentile(data_sim, 25)
    q75_1 = np.nanpercentile(data_sim, 75)
    q10_1 = np.nanpercentile(data_sim, 10)
    q90_1 = np.nanpercentile(data_sim, 90)
    err1_low = med1 - q10_1
    err1_high = q90_1 - med1
    # DA
    med2,q25_2,q75_2, err2_low, err2_high = get_stats(data_da)
    x = np.arange(np.shape(data_da)[1]+1)
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=1.5)
    bar_width = 0.4
    ax.bar(x[0],q75_1-q25_1, bottom=q25_1, capsize=5, color='blue', width=bar_width, edgecolor='gray')
    ax.bar(x[1:], q75_2-q25_2, bottom=q25_2, capsize=5, color='red', width=bar_width, edgecolor='gray')
    for i in range(len(x)-1):
        ax.plot(x[i+1], med2[i], marker='_', color='black', markersize=12, linewidth=150, zorder=3)
    ax.plot(x[0], med1, marker='_', color='black', markersize=12, linewidth=150, zorder=3)
    ax.set_xticks(range(np.shape(data_da)[1]+1))
    if 'ils01' in pm_path:
        ax.set_xticklabels(['sim']+[str(i) for i in range(1, np.shape(data_da)[1]+1)])
    if 'ils03' in pm_path:
        ax.set_xticklabels(['sim','1-3','4-6','7-9','10-12','13-15','16-18','19-21','22-24'])
    if 'ils06' in pm_path:
        ax.set_xticklabels(['sim','1-6','7-12','13-18','19-24'])
    if 'ils12' in pm_path:
        ax.set_xticklabels(['sim','1-12','13-24'])
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(-0.5,np.shape(data_da)[1]+1-0.5)
    ax.set_ylabel(ylabel,fontsize=12)
    ax.set_xlabel('Hours after DA (h)',fontsize=12)


fig, axs = plt.subplots(1, 3, figsize=(18, 4),dpi=600)
plot_bar_error(axs[0],metric_sim[:,0],rmse_da,0.6,2.4,'RMSE (m)')
plot_bar_error(axs[1],metric_sim[:,1],bias_da,0,2.1,'Bias (m)')
plot_bar_error(axs[2],time_error_sim,time_error_da,-5,8,'Timing error (h)')
plt.tight_layout()
plt.savefig(exp_plotdir+'4.metric.jpg', format='jpg',dpi=600)
plt.show()

In [ ]:
#%% evaluate the 24 hours forecast performance which is ahead of peak flow
def evaluate_24h(wlv_da,sim_point_ind,obs_point_ind):
    # [stations,ahead_hours,five_vars]
    da_matrix  = np.full((np.shape(obs_point_ind)[0],int(24/dahour),5),np.nan) # from 1 to 24 hours ahead
    sim_matrix = np.full((np.shape(obs_point_ind)[0],int(24/dahour),5),np.nan) # from 1 to 24 hours ahead
    for station_ind in range(0,np.shape(obs_point_ind)[0]):
        obs_data = obs_point_ind[station_ind,:]
        obs_peak_ind = np.argmax(obs_data) + tstart_da
        obs_peak = np.argmax(obs_data) + tstart
        obs_peak1 = math.ceil((obs_peak-24)/dahour)
        obs_peak2 = math.floor(obs_peak/dahour)
        da_data  = np.nanmean(wlv_da[:,obs_peak1:obs_peak2,station_ind,:],axis=0)
        # 1-24 hours ahead
        if obs_peak1-tstart_da<0:
            continue
        else:
            for t_ahead in range(1,1+int(24/dahour)):
                sim_use = np.nanmean(sim_point_ind[station_ind,obs_peak-tstart-t_ahead:obs_peak-tstart-t_ahead+24,:],axis=1)
                obs_use = obs_point_ind[station_ind,obs_peak-tstart-t_ahead:obs_peak-tstart-t_ahead+24]
                da_use  = da_data[obs_peak2-obs_peak1-t_ahead,:]
                # if (station_ind == 7) & ((t_ahead == 1) | (t_ahead == 5)):
                #     print(t_ahead,sim_use,obs_use,da_use)
                da_matrix[station_ind,t_ahead-1,0],da_matrix[station_ind,t_ahead-1,1],da_matrix[station_ind,t_ahead-1,2],da_matrix[station_ind,t_ahead-1,3],da_matrix[station_ind,t_ahead-1,4]  = cal_single_metric(obs_use,da_use)
                sim_matrix[station_ind,t_ahead-1,0],sim_matrix[station_ind,t_ahead-1,1],sim_matrix[station_ind,t_ahead-1,2],sim_matrix[station_ind,t_ahead-1,3],sim_matrix[station_ind,t_ahead-1,4] = cal_single_metric(obs_use,sim_use)
    # rmse,bias,kge,time_error,pearson_r
    return da_matrix,sim_matrix
da_matrix_wlv,sim_matrix_wlv = evaluate_24h(wlv_da,sim_point_ind,obs_point_ind)
da_matrix_dis,sim_matrix_dis = evaluate_24h(dis_da_use,sim_point_dis,obs_point_dis)
        

In [ ]:
# plot bar evaluation for 24 hours forecast
def plot_bar_24fore(ax,data_sim,data_da,ymin,ymax,ylabel):
    def get_stats(data):
        median = np.nanpercentile(data, 50, axis=0)
        q25 = np.nanpercentile(data, 25, axis=0)
        q75 = np.nanpercentile(data, 75, axis=0)
        q10 = np.nanpercentile(data, 10, axis=0)
        q90 = np.nanpercentile(data, 90, axis=0)
        err_lower = median - q10
        err_upper = q90 - median
        return median,q25,q75,err_lower,err_upper
    # sim
    med1,q25_1,q75_1, err1_low, err1_high = get_stats(data_sim)
    # DA
    med2,q25_2,q75_2, err2_low, err2_high = get_stats(data_da)
    x = np.arange(np.shape(data_da)[1])
    ax.axhline(y=0, color='gray', linestyle='--', linewidth=1.5)
    bar_width = 0.4
    ax.bar(x-bar_width/2,q75_1-q25_1, bottom=q25_1, capsize=5, color='blue', width=bar_width, edgecolor='gray')
    ax.bar(x+bar_width/2, q75_2-q25_2, bottom=q25_2, capsize=5, color='red', width=bar_width, edgecolor='gray')
    for i in range(len(x)):
        ax.plot(x[i]-bar_width/2, med1[i], marker='_', color='black', markersize=12, linewidth=150, zorder=3)
        ax.plot(x[i]+bar_width/2, med2[i], marker='_', color='black', markersize=12, linewidth=150, zorder=3)
    ax.set_xticks(np.arange(0,np.shape(data_da)[1]))
    if 'ils01' in pm_path or 'round' in pm_path:
        ax.set_xticklabels([str(i) for i in range(1, 25)])
    if 'ils03' in pm_path:
        ax.set_xticklabels(['1-3','4-6','7-9','10-12','13-15','16-18','19-21','22-24'])
    if 'ils06' in pm_path:
        ax.set_xticklabels(['1-6','7-12','13-18','19-24'])
    if 'ils12' in pm_path:
        ax.set_xticklabels(['1-12','13-24'])
    ax.set_ylim(ymin,ymax)
    ax.set_xlim(-0.5,np.nanmax(x)+0.5)
    ax.set_ylabel(ylabel,fontsize=12)
    ax.set_xlabel('Forecast Leading Time Before Peak Flow (h)',fontsize=12)

def plot_24matirx(sim_matrix_wlv,da_matrix_wlv,y1,y2,y3,y4,y5,y6,y7,y8,varname):
    fig, axs = plt.subplots(2, 2, figsize=(16, 8),dpi=600)
    if varname == 'wlv':
        plot_bar_24fore(axs[0,0],sim_matrix_wlv[:,:,0],da_matrix_wlv[:,:,0],y1,y2,'RMSE (m)')
        plot_bar_24fore(axs[0,1],sim_matrix_wlv[:,:,1],da_matrix_wlv[:,:,1],y3,y4,'Bias (m)')
    else:
        plot_bar_24fore(axs[0,0],sim_matrix_wlv[:,:,0],da_matrix_wlv[:,:,0],y1,y2,'RMSE (m${^3}$/s)')
        plot_bar_24fore(axs[0,1],sim_matrix_wlv[:,:,1],da_matrix_wlv[:,:,1],y3,y4,'Bias (m${^3}$/s)')
    plot_bar_24fore(axs[1,0],sim_matrix_wlv[:,:,2],da_matrix_wlv[:,:,2],y5,y6,'KGE')
    plot_bar_24fore(axs[1,1],sim_matrix_wlv[:,:,4],da_matrix_wlv[:,:,4],y7,y8,'Pearson Correlation Coefficient')
    plt.tight_layout()
    plt.savefig(exp_plotdir+'5.24hour_metric_'+varname+'.jpg', format='jpg',dpi=600)
    plt.show()
if 'ils01' in pm_path or 'round' or 'msm' in pm_path:
    plot_24matirx(sim_matrix_wlv,da_matrix_wlv,0.5,4.,-1.6,3.8,-5,0.8,-0.,1,'wlv')
    plot_24matirx(sim_matrix_dis,da_matrix_dis,80,2600,-1100,2200,-7.2,0.8,-0.,1,'dis')
if 'ils03' in pm_path:
    plot_24matirx(sim_matrix_wlv,da_matrix_wlv,0.3,3.5,-0.6,3.2,-3,0.8,-0.8,1,'wlv')
    plot_24matirx(sim_matrix_dis,da_matrix_dis,0,750,-250,500,-3,0.8,-0.8,1,'dis')

In [ ]:
#%% plot flood extend
filepath = '/data50/yingying/flood/'
def read_cov_tif(filename):
    filen = filepath + filename
    with rasterio.open(filen) as dataset:
        tiff_data = dataset.read(1)
        # tiff_data = tiff_data[::-1,:]
        tiff_data = np.where(tiff_data==0,np.nan,tiff_data)
        col = dataset.width      
        row = dataset.height     
        bounds = dataset.bounds
        min_lon, min_lat, max_lon, max_lat = bounds
        print(f"Longitude Range: {min_lon} to {max_lon}")
        print(f"Latitude Range: {min_lat} to {max_lat}")
        print('--------')
    return col,row,min_lon,max_lon,min_lat,max_lat,tiff_data
# 20191012T2042 UST
col1,row1,min_lon1,max_lon1,min_lat1,max_lat1,fld_reg1 = read_cov_tif('area_1_result.tif')  # 138-138.4, 36.6-36.8
col2,row2,min_lon2,max_lon2,min_lat2,max_lat2,fld_reg2 = read_cov_tif('area_2_result.tif')  # 139.3-139.8, 34.45-35.7

# col1,row1,min_lon1,max_lon1,min_lat1,max_lat1,fld_reg1 = read_cov_tif('binary_output_without_water_area2.tif')  # 137.4-140.6, 35.9-37.8
# col2,row2,min_lon2,max_lon2,min_lat2,max_lat2,fld_reg2 = read_cov_tif('binary_output_without_water_coast.tif')  # 138.1-141.2, 34.5-36.4

lon_map1 = np.linspace(min_lon1,max_lon1,col1)
lat_map1 = np.linspace(min_lat1,max_lat1,row1)
lon2d_map1, lat2d_map1 = np.meshgrid(lon_map1, lat_map1)
lon_map2 = np.linspace(min_lon2,max_lon2,col2)
lat_map2 = np.linspace(min_lat2,max_lat2,row2)
lon2d_map2, lat2d_map2 = np.meshgrid(lon_map2, lat_map2)

In [ ]:
# Figure6: SAR map
font_label = {'family': 'serif', 'weight': 'normal', 'size': 10.5}
def draw_sat_fld():
    fig = plt.figure(dpi = 600,figsize=(6,6))
    ax1 = fig.add_axes([0.1,0.1,0.8,0.8],projection=ccrs.PlateCarree())
    ax1.set_axis_off()
    ax1.set_extent([lon_min,lon_max,lat_min,lat_max], ccrs.PlateCarree())
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=0, linestyle='-.')
    ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
    # mask the other place
    region_lon1 = [128.06, 128.06,138, 138 ]
    region_lat1 = [41 , 45.95 , 45.95, 41  ]
    region_lon2 = [128.06,128.06,131.4,131.4 ]
    region_lat2 = [34.5,41,41,34.5]
    ax1.fill(region_lon1, region_lat1, color='white', transform=ccrs.PlateCarree(), zorder=10)
    ax1.fill(region_lon2, region_lat2, color='white', transform=ccrs.PlateCarree(), zorder=10)
    cmap = ListedColormap(['blue']) 
    gl.top_labels   = False
    gl.right_labels = False
    gl.bottom_labels   = False
    gl.left_labels = False
    lon2d_tiff, lat2d_tiff = np.meshgrid(lon_tiff, lat_tiff)
    uparea_reg = np.where(np.isnan(uparea_low),np.nan,0.002)
    ax1.scatter(lon2d_tiff, lat2d_tiff,uparea_reg, c='blue',zorder=2, transform=ccrs.PlateCarree())
    # con_plot1 = ax1.pcolormesh(lon_tiff,lat_tiff,uparea_low,zorder=2, cmap=cmap,alpha=0.35, transform=ccrs.PlateCarree(),vmin=0,vmax=30)  
    def draw_sat_reg(lon2d_map,lat2d_map,data):
        # control the size for the flooding area
        ax1.scatter(lon2d_map, lat2d_map, data, color='black', zorder=3, transform=ccrs.PlateCarree())
    draw_sat_reg(lon2d_map1,lat2d_map1,fld_reg1[::-1,:])
    draw_sat_reg(lon2d_map2,lat2d_map2,fld_reg2[::-1,:])
    plt.savefig(inputdir+'6_SARmap.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
# draw_sat_fld()

In [ ]:
#%% read flood extend data
# read sim/DA output
def read_flood(dhour):
    da_data = []
    sim_data = []
    SAR_time = datetime(2019,10,12,21)
    time_step = timedelta(hours=dhour)
    ctime = SAR_time - time_step
    cyyyy='%04d' % (ctime.year)
    cmm='%02d' % (ctime.month)
    cdd='%02d' % (ctime.day)
    chh='%02d' % (ctime.hour)
    ctime = cyyyy + cmm + cdd + chh
    for ens in range(0,pm.ens_mem()):
        ens_str = '{:03d}'.format(ens+1)
        if os.path.exists(outdir+'/CaMa_out_'+expname+dahour_str+'/'+ctime+'A'+ens_str+'/o_fldfrc'+syyyy+'.nc'):
            fname_da  = outdir+'/CaMa_out_'+expname+dahour_str+'/'+ctime+'A'+ens_str+'/o_fldfrc'+syyyy+'.nc'
            fname_sim = outdir+'/CaMa_out_'+expname+dahour_str+'/'+ctime+'C'+ens_str+'/o_fldfrc'+syyyy+'.nc'
        else:
            chour = math.floor(dahour)
            chh = '{:02d}'.format(chour)
            ctime = cyyyy + cmm + cdd + chh
            fname_da  = outdir+'/CaMa_out_'+expname+dahour_str+'/'+ctime+'A'+ens_str+'/o_fldfrc'+syyyy+'.nc'
            fname_sim = outdir+'/CaMa_out_'+expname+dahour_str+'/'+ctime+'C'+ens_str+'/o_fldfrc'+syyyy+'.nc'
        flood_da  = read_nc(fname_da,3)
        flood_sim = read_nc(fname_sim,3)
        lon_region_ind = np.where((lon>=lon_min)&(lon<=lon_max))[0]
        lon_region = lon[lon_region_ind]
        lat_region_ind = np.where((lat>=lat_min)&(lat<=lat_max))[0]
        lat_region = lat[lat_region_ind]
        flood_da_region  = flood_da[dhour,lat_region_ind[:,None],lon_region_ind]
        flood_sim_region = flood_sim[dhour,lat_region_ind[:,None],lon_region_ind]
        da_data.append(flood_da_region)
        sim_data.append(flood_sim_region)
        break
    # da_all = np.nanmean(np.stack(da_data,axis=0),axis=0)
    # sim_all = np.nanmean(np.stack(sim_data,axis=0),axis=0)
    # print(np.shape(da_all)) # [180,240]
    return lon_region,lat_region
    
# [lat,lon,time_ahead]
lon_region,lat_region = read_flood(21)
# flood_da = np.full((len(lat_region),len(lon_region),23),np.nan)
# flood_sim = np.full((len(lat_region),len(lon_region),23),np.nan)
# for dhour in range(1,24):
#     flood_da_each,flood_sim_each,lon_region,lat_region = read_flood(dhour)
#     flood_da[:,:,dhour-1] = flood_da_each
#     flood_sim[:,:,dhour-1] = flood_sim_each
    

filepath = inputdir+'/exp_'+expname+dahour_str+'/'
# np.save(filepath+'flood'+'/flood_da.npy',flood_da)
# np.save(filepath+'flood'+'/flood_sim.npy',flood_sim)
flood_da = np.load(filepath+'flood'+'/flood_da.npy')
flood_sim = np.load(filepath+'flood'+'/flood_sim.npy')

def find_sim_region(min_lon,max_lon,min_lat,max_lat):
    lat_ind = np.where((lat_region>=min_lat)&(lat_region<=max_lat))[0]
    lon_ind = np.where((lon_region>=min_lon)&(lon_region<=max_lon))[0]
    lat_reg = lat_region[np.nanmin(lat_ind):np.nanmax(lat_ind)+1]
    lon_reg = lon_region[np.nanmin(lon_ind):np.nanmax(lon_ind)+1]
    lon2d_map, lat2d_map = np.meshgrid(lon_reg, lat_reg)
    sim_fld_reg = flood_sim[np.nanmin(lat_ind):np.nanmax(lat_ind)+1,np.nanmin(lon_ind):np.nanmax(lon_ind)+1,:]
    da_fld_reg  = flood_da[np.nanmin(lat_ind):np.nanmax(lat_ind)+1,np.nanmin(lon_ind):np.nanmax(lon_ind)+1,:]
    return lon2d_map, lat2d_map,sim_fld_reg,da_fld_reg
lon2d_sim1,lat2d_sim1,sim_fld_reg1,da_fld_reg1 = find_sim_region(min_lon1,max_lon1,min_lat1,max_lat1)
lon2d_sim2,lat2d_sim2,sim_fld_reg2,da_fld_reg2 = find_sim_region(min_lon2,max_lon2,min_lat2,max_lat2)

In [ ]:
# upscaling for SAR data
window_size = 46   #93 grids, depends on resolution
def upscale_sat(fld_reg,sim_fld_reg,da_fld_reg,exp_type):
    fld_upscale = np.full((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1]),np.nan)
    for i in range(0,np.shape(sim_fld_reg)[0]): # lat
        for j in range(0,np.shape(sim_fld_reg)[1]): # lon
            sim = sim_fld_reg[i,j]
            da  = da_fld_reg[i,j]
            lat_i1 = i*window_size*2-window_size
            lat_i2 = i*window_size*2+window_size
            lon_i1 = j*window_size*2-window_size
            lon_i2 = j*window_size*2+window_size
            if lat_i1<0:
                lat_i1 = 0
            if lon_i1<0:
                lon_i1 = 0            
            if lat_i2>np.shape(fld_reg)[0]:
                lat_i2 = np.shape(fld_reg)[0]    
            if lon_i2>np.shape(fld_reg)[1]:
                lon_i2 = np.shape(fld_reg)[1]
            # print(lat_i1,lat_i2,lon_i1,lon_i2]))
            fld_data = fld_reg[lat_i1:lat_i2,lon_i1:lon_i2]
            if exp_type == 'weight':
                fld_upscale[i,j] = np.nansum(fld_data)/((lat_i2-lat_i1)*(lon_i2-lon_i1))
            else:
                if np.nansum(fld_data)>0:
                    fld_upscale[i,j] = 1
                else:
                    fld_upscale[i,j] = 0
    return fld_upscale
fld_up1 = upscale_sat(fld_reg1,sim_fld_reg1,da_fld_reg1,'weight')
fld_up2 = upscale_sat(fld_reg2,sim_fld_reg2,da_fld_reg2,'weight')
fld_grid1 = upscale_sat(fld_reg2,sim_fld_reg2,da_fld_reg2,'grid')
fld_grid2 = upscale_sat(fld_reg2,sim_fld_reg2,da_fld_reg2,'grid')

In [ ]:
# Figure6: sim flooding map
lon_ret1, lon_ret2,lat_ret1, lat_ret2 = 138,138.41,36.6,36.8
lon_ret3, lon_ret4,lat_ret3, lat_ret4 = 139.28,139.83,35.44,35.72
# lon_ret3, lon_ret4,lat_ret3, lat_ret4 = 139.4,140,35.5,36.3
def draw_sat_fld():
    fig = plt.figure(dpi = 600,figsize=(6,6))
    ax1 = fig.add_axes([0.1,0.1,0.8,0.8],projection=ccrs.PlateCarree())
    ax1.set_axis_off()
    ax1.set_extent([136.5,141,34,37], ccrs.PlateCarree())
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=0, linestyle='-.')
    ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
    # mask the other place
    region_lon1 = [128.06, 128.06,138, 138 ]
    region_lat1 = [41 , 45.95 , 45.95, 41  ]
    region_lon2 = [128.06,128.06,131.4,131.4 ]
    region_lat2 = [34.5,41,41,34.5]
    ax1.fill(region_lon1, region_lat1, color='white', transform=ccrs.PlateCarree(), zorder=10)
    ax1.fill(region_lon2, region_lat2, color='white', transform=ccrs.PlateCarree(), zorder=10)
    cmap = ListedColormap(['blue']) 
    gl.top_labels   = False
    gl.right_labels = False
    # identify region
    # 138-138.4, 36.6-36.8
    # 139.3-139.8, 35.45-35.7
    def add_rect(lon_min, lon_max,lat_min,lat_max):
        rect = Rectangle((lon_min,lat_min), lon_max-lon_min, lat_max-lat_min, linewidth=0.5, edgecolor='black',facecolor='none',transform=ccrs.PlateCarree())
        ax1.add_patch(rect)
    add_rect(lon_ret1, lon_ret2,lat_ret1, lat_ret2)
    add_rect(lon_ret3, lon_ret4,lat_ret3, lat_ret4)
    lon2d_tiff, lat2d_tiff = np.meshgrid(lon_tiff, lat_tiff)
    uparea_reg = np.where(np.isnan(uparea_low),np.nan,0.01)
    ax1.scatter(lon2d_tiff, lat2d_tiff,uparea_reg, c='blue',zorder=2, transform=ccrs.PlateCarree())
    # con_plot1 = ax1.pcolormesh(lon_tiff,lat_tiff,uparea_low,zorder=2, cmap=cmap,alpha=0.35, transform=ccrs.PlateCarree(),vmin=0,vmax=30)  
    fld_up_plot1 = np.where(np.isnan(fld_up1)|(fld_up1<10**(-8)),np.nan,0.5)
    fld_up_plot2 = np.where(np.isnan(fld_up2)|(fld_up2<10**(-8)),np.nan,0.5)
    def draw_sat_reg(lon2d_map,lat2d_map,data,c):
        # control the size for the flooding area
        ax1.scatter(lon2d_map, lat2d_map, data, color=c, zorder=3, transform=ccrs.PlateCarree(),marker='.')
    draw_sat_reg(lon2d_sim1,lat2d_sim1,sim_fld_reg1[:,:,2],'orange')
    draw_sat_reg(lon2d_sim1,lat2d_sim1,da_fld_reg1[:,:,2],'red')
    draw_sat_reg(lon2d_sim2,lat2d_sim2,sim_fld_reg2[:,:,2],'orange')
    draw_sat_reg(lon2d_sim2,lat2d_sim2,da_fld_reg2[:,:,2],'red')
    draw_sat_reg(lon2d_sim1,lat2d_sim1,fld_up_plot1,'black')
    draw_sat_reg(lon2d_sim2,lat2d_sim2,fld_up_plot2,'black')
    # draw_sat_reg(lon2d_map1,lat2d_map1,fld_reg1[::-1,:]*0.05,'black')
    # draw_sat_reg(lon2d_map2,lat2d_map2,fld_reg2[::-1,:]*0.05,'black')
    plt.savefig(exp_plotdir+'6.24hour_flood_map.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
draw_sat_fld()

In [ ]:
# region compare
def cal_comp_fld(fld_up,sim_fld_reg,da_fld_reg):
    cal_sim_fld = np.zeros((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1]))
    cal_da_fld  = np.zeros((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1]))
    cal_fld     = np.full((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1]),np.nan)
    size_fld    = np.full((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1]),np.nan)
    for i in range(0,np.shape(sim_fld_reg)[0]): # lat
        for j in range(0,np.shape(sim_fld_reg)[1]): # lon
            # simulation condition
            if (fld_up[i,j]<10**(-8)) & (sim_fld_reg[i,j]>10**(-8)):  # only sim
                cal_sim_fld[i,j] = 1
            if (fld_up[i,j]>10**(-8)) & (sim_fld_reg[i,j]>10**(-8)):  # both sim and sat
                cal_sim_fld[i,j] = 2
            if (fld_up[i,j]>10**(-8)) & (sim_fld_reg[i,j]<10**(-8)):  # only sat
                cal_sim_fld[i,j] = 3
            # da condition
            if (fld_up[i,j]<10**(-8)) & (da_fld_reg[i,j]>10**(-8)):  # only da
                cal_da_fld[i,j] = 1
            if (fld_up[i,j]>10**(-8)) & (da_fld_reg[i,j]>10**(-8)):  # both da and sat
               cal_da_fld[i,j] = 2
            if (fld_up[i,j]>10**(-8)) & (da_fld_reg[i,j]<10**(-8)):  # only sat
                cal_da_fld[i,j] = 3
            # 0.5: only sim, 1.5: only da, 2.5: only sat, 3.5: sim+da, 4.5: sim+sat, 5.5: da+sat, 6.5: sim+da+sat, nan: no flood
            if (cal_sim_fld[i,j]==1) & (cal_da_fld[i,j]==0):
                cal_fld[i,j] = 0.5
                size_fld[i,j]= sim_fld_reg[i,j]*100
            if (cal_sim_fld[i,j]==0) & (cal_da_fld[i,j]==1):
                cal_fld[i,j] = 1.5
                size_fld[i,j]= da_fld_reg[i,j]*100
            if (cal_sim_fld[i,j]==3) & (cal_da_fld[i,j]==3):
                cal_fld[i,j] = 2.5                
                size_fld[i,j]= fld_up[i,j]*100
            if (cal_sim_fld[i,j]==1) & (cal_da_fld[i,j]==1):
                cal_fld[i,j] = 3.5            
                size_fld[i,j]= sim_fld_reg[i,j]*100
            if (cal_sim_fld[i,j]==2) & (cal_da_fld[i,j]==3):
                cal_fld[i,j] = 4.5                
                size_fld[i,j]= fld_up[i,j]*100
            if (cal_sim_fld[i,j]==3) & (cal_da_fld[i,j]==2):
                cal_fld[i,j] = 5.5
                size_fld[i,j]= fld_up[i,j]*100
            if (cal_sim_fld[i,j]==2) & (cal_da_fld[i,j]==2):
                cal_fld[i,j] = 6.5      
                size_fld[i,j]= fld_up[i,j]*100
    def cal_csi(cal_sim_fld,sim_fld_reg):
        sim_bottom = 0
        sim_up = 0
        for i in range(0,np.shape(sim_fld_reg)[0]): # lat
            for j in range(0,np.shape(sim_fld_reg)[1]): # lon
                if cal_sim_fld[i,j] == 1:
                    sim_bottom = sim_bottom + sim_fld_reg[i,j]
                if cal_sim_fld[i,j] == 2:
                    sim_up = sim_up + fld_up[i,j]
                    sim_bottom = sim_bottom + np.abs(sim_fld_reg[i,j]-fld_up[i,j])
                    # print(sim_fld_reg[i,j],fld_up[i,j],sim_fld_reg[i,j])
                if cal_sim_fld[i,j] == 3:
                    sim_bottom = sim_bottom + fld_up[i,j]
        return sim_up/sim_bottom
    csi_sim = cal_csi(cal_sim_fld,sim_fld_reg)
    csi_da  = cal_csi(cal_da_fld,da_fld_reg)
    print('score sim:','{:.2f}'.format(csi_sim),'score_da:','{:.2f}'.format(csi_da))
    return cal_fld,size_fld,cal_sim_fld,cal_da_fld,csi_sim,csi_da

# from forecast time ahead 1hour to 24hours
def evaluate_flood(fld_up,sim_fld_reg,da_fld_reg):
    cal_sim_fld = np.zeros((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1],np.shape(sim_fld_reg)[2]))
    cal_da_fld  = np.zeros((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1],np.shape(sim_fld_reg)[2]))
    cal_fld     = np.full((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1],np.shape(sim_fld_reg)[2]),np.nan)
    size_fld    = np.full((np.shape(sim_fld_reg)[0],np.shape(sim_fld_reg)[1],np.shape(sim_fld_reg)[2]),np.nan)
    csi_sim = np.full(np.shape(sim_fld_reg)[2],np.nan)
    csi_da  = np.full(np.shape(sim_fld_reg)[2],np.nan)
    for i in range(0,np.shape(sim_fld_reg)[2]):
        cal_fld[:,:,i],size_fld[:,:,i],cal_sim_fld[:,:,i],cal_da_fld[:,:,i],csi_sim[i],csi_da[i] = cal_comp_fld(fld_up,sim_fld_reg[:,:,i],da_fld_reg[:,:,i])
    return cal_fld,size_fld,cal_sim_fld,cal_da_fld,csi_sim,csi_da
cal_fld1,size_fld1,cal_sim_fld1,cal_da_fld1,csi_sim1,csi_da1 = evaluate_flood(fld_up1,sim_fld_reg1,da_fld_reg1)
cal_fld2,size_fld2,cal_sim_fld2,cal_da_fld2,csi_sim2,csi_da2 = evaluate_flood(fld_up2,sim_fld_reg2,da_fld_reg2)
print('reg1:',np.argmax(csi_da1-csi_sim1))
print('reg2:',np.argmax(csi_da2-csi_sim2))

In [ ]:
def plot_flood_condition(csi_da1,csi_sim1,csi_da2,csi_sim2):
    fig = plt.figure(dpi = 600,figsize=(6,6))
    ax1 = fig.add_axes([0.1,0.65,0.7,0.35],projection=ccrs.PlateCarree())
    ax2 = fig.add_axes([0.1,0.26,0.7,0.38],projection=ccrs.PlateCarree())
    ax3 = fig.add_axes([0.1,0.1,0.7,0.15])
    def comp_fld(ax1, lon_ret1, lon_ret2,lat_ret1, lat_ret2, lon2d_sim, lat2d_sim, cal_fld,size_fld):
        ax1.set_extent([lon_ret1, lon_ret2,lat_ret1, lat_ret2], ccrs.PlateCarree())
        gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                        linewidth=1, color='gray', alpha=0, linestyle='-.')
        ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
        cmap_blue = ListedColormap(['blue']) 
        colors = plt.cm.rainbow(np.linspace(0, 1, 7)) 
        cmap = ListedColormap(colors) 
        bounds = np.arange(0, 8)
        norm = BoundaryNorm(bounds, cmap.N) 
        gl.top_labels   = False
        gl.right_labels = False
        gl.bottom_labels= False
        gl.left_labels = False
        lon_map = np.linspace(lon_min,lon_max,int((lon_max-lon_min)*3600))
        lat_map = np.linspace(lat_min,lat_max,int((lat_max-lat_min)*3600))
        lon_tiff_ind = np.where((lon_map>=lon_ret1)&(lon_map<=lon_ret2))[0]
        lat_tiff_ind = np.where((lat_map>=lat_ret1)&(lat_map<=lat_ret2))[0]
        lat_tiff_reg = lat_map[np.nanmin(lat_tiff_ind):np.nanmax(lat_tiff_ind)+1]
        lon_tiff_reg = lon_map[np.nanmin(lon_tiff_ind):np.nanmax(lon_tiff_ind)+1]
        lon2d_tiff, lat2d_tiff = np.meshgrid(lon_tiff_reg, lat_tiff_reg)
        uparea_reg = up_area[np.nanmin(lat_tiff_ind):np.nanmax(lat_tiff_ind)+1,np.nanmin(lon_tiff_ind):np.nanmax(lon_tiff_ind)+1]
        uparea_reg = np.where(uparea_reg<10,np.nan,0.01)
        ax1.scatter(lon2d_tiff, lat2d_tiff,uparea_reg, c='blue',zorder=3, transform=ccrs.PlateCarree())
        ## ax1.pcolormesh(lon_tiff_reg,lat_tiff_reg,uparea_reg,zorder=2, cmap=cmap_blue, transform=ccrs.PlateCarree())  
        con_plot2 = ax1.scatter(lon2d_sim, lat2d_sim, c=cal_fld, s=size_fld,cmap=cmap, norm=norm, zorder=3, transform=ccrs.PlateCarree())
        def color_bar(l,b,w,h):
          rect = [l,b,w,h]
          cbar_ax = fig.add_axes(rect)
          return cbar_ax
        [l2,b2,w2,h2] = [0.81,0.38,0.03,0.5]
        cbar_ax2 = color_bar(l2,b2,w2,h2)
        cb2 = plt.colorbar(con_plot2, cax=cbar_ax2,orientation="vertical",shrink=0.5)
        cb2.set_ticks([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])
        cb2.set_ticklabels(['only Sim','only DA','only SAR','Sim+DA','Sim+SAR','DA+SAR','Sim+DA+SAR'],fontsize=8)
    # 0.5: only sim, 1.5: only da, 2.5: only sat, 3.5: sim+da, 4.5: sim+sat, 5.5: da+sat, 6.5: sim+da+sat, nan: no flood
    comp_fld(ax1,lon_ret1, lon_ret2,lat_ret1, lat_ret2, lon2d_sim1, lat2d_sim1, cal_fld1[:,:,0], size_fld1[:,:,0])
    ax1.text(0.01, 0.95, 'Reg1',transform=ax1.transAxes,fontsize=10,verticalalignment='top', horizontalalignment='left',fontweight='bold')
    comp_fld(ax2,lon_ret3, lon_ret4,lat_ret3, lat_ret4, lon2d_sim2, lat2d_sim2, cal_fld2[:,:,0], size_fld2[:,:,0])
    ax2.text(0.01, 0.95, 'Reg2',transform=ax2.transAxes,fontsize=10,verticalalignment='top', horizontalalignment='left',fontweight='bold')
    x = np.arange(1,1+np.shape(flood_da)[2],1)
    ax3.plot(x,csi_da1,'r-',label='DA(Reg1)',lw=1.5)
    ax3.plot(x,csi_sim1,'b-',label='Sim(Reg1)',lw=1.5)
    ax3.plot(x,csi_da2,color='red',linestyle='--',label='DA(Reg2)',lw=1.5)
    ax3.plot(x,csi_sim2,color='blue',linestyle='--',label='Sim(Reg2)',lw=1.5)
    ax3.legend(loc='upper center', bbox_to_anchor=(0.55, 0.97), ncol=4, fontsize=7, frameon=False)
    ax3.set_xlim(0.5,0.5+np.shape(flood_da)[2])
    ax3.set_ylim(0,0.25)
    ax3.set_ylabel('CSI',fontsize=10)
    if np.shape(flood_da)[2]>20:
        ax3.set_xticks(np.arange(1,1+np.shape(flood_da)[2],2))
        ax3.set_xticklabels(np.arange(1,1+np.shape(flood_da)[2],2))
    else:
        ax3.set_xticks(np.arange(1,1+np.shape(flood_da)[2]))
    if 'ils03' in pm_path:
        ax3.set_xticklabels(['3','6','9','12','15','18','21'])
    if 'ils06' in pm_path:
        ax3.set_xticklabels(['3','9','15','21'])
    if 'ils12' in pm_path:
        ax3.set_xticklabels(['9','21'])
    ax3.set_xlabel('DA Time Before SAR Observation (h)',fontsize=10)
    plt.savefig(exp_plotdir+'6.24hour_flood_region.jpg', format='jpg',dpi=600)
    plt.show()
    plt.close()
plot_flood_condition(csi_da1,csi_sim1,csi_da2,csi_sim2)
plot_flood_condition(csi_da_grid1,csi_sim_grid1,csi_da_grid2,csi_sim_grid2)

In [ ]:
#%% plot flood condition for 1-24 hours ahead
def plot_flood_condition_all():
    fig, axes = plt.subplots(8, 6, figsize=(36, 24), dpi=600,subplot_kw={'projection': ccrs.PlateCarree()})
    axes = axes.flatten()
    def comp_fld(ax1, lon_ret1, lon_ret2,lat_ret1, lat_ret2, lon2d_sim, lat2d_sim, cal_fld, size_fld, plot_idx):
        ax1.set_extent([lon_ret1, lon_ret2,lat_ret1, lat_ret2], ccrs.PlateCarree())
        gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                        linewidth=1, color='gray', alpha=0, linestyle='-.')
        ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
        cmap_blue = ListedColormap(['blue']) 
        colors = plt.cm.rainbow(np.linspace(0, 1, 7)) 
        cmap = ListedColormap(colors) 
        bounds = np.arange(0, 8)
        norm = BoundaryNorm(bounds, cmap.N) 
        gl.top_labels   = False
        gl.right_labels = False
        gl.bottom_labels= False
        gl.left_labels = False
        lon_map = np.linspace(lon_min,lon_max,int((lon_max-lon_min)*3600))
        lat_map = np.linspace(lat_min,lat_max,int((lat_max-lat_min)*3600))
        lon_tiff_ind = np.where((lon_map>=lon_ret1)&(lon_map<=lon_ret2))[0]
        lat_tiff_ind = np.where((lat_map>=lat_ret1)&(lat_map<=lat_ret2))[0]
        lat_tiff_reg = lat_map[np.nanmin(lat_tiff_ind):np.nanmax(lat_tiff_ind)+1]
        lon_tiff_reg = lon_map[np.nanmin(lon_tiff_ind):np.nanmax(lon_tiff_ind)+1]
        lon2d_tiff, lat2d_tiff = np.meshgrid(lon_tiff_reg, lat_tiff_reg)
        uparea_reg = up_area[np.nanmin(lat_tiff_ind):np.nanmax(lat_tiff_ind)+1,np.nanmin(lon_tiff_ind):np.nanmax(lon_tiff_ind)+1]
        uparea_reg = np.where(uparea_reg<10,np.nan,0.01)
        ax1.scatter(lon2d_tiff, lat2d_tiff,uparea_reg, c='blue',zorder=3, transform=ccrs.PlateCarree())
        ## ax1.pcolormesh(lon_tiff_reg,lat_tiff_reg,uparea_reg,zorder=2, cmap=cmap_blue, transform=ccrs.PlateCarree())  
        con_plot2 = ax1.scatter(lon2d_sim, lat2d_sim, c=cal_fld, s=size_fld,cmap=cmap, norm=norm, zorder=3, transform=ccrs.PlateCarree())
        if plot_idx == 0:
            def color_bar(l,b,w,h):
              rect = [l,b,w,h]
              cbar_ax = fig.add_axes(rect)
              return cbar_ax
            [l2,b2,w2,h2] = [0.85,0.28,0.03,0.75]
            cbar_ax2 = color_bar(l2,b2,w2,h2)
            cb2 = plt.colorbar(con_plot2, cax=cbar_ax2,orientation="vertical",shrink=0.5)
            cb2.set_ticks([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5])
            cb2.set_ticklabels(['only Sim','only DA','only SAR','Sim+DA','Sim+SAR','DA+SAR','Sim+DA+SAR'],fontdict=font_label)
    # 0.5: only sim, 1.5: only da, 2.5: only sat, 3.5: sim+da, 4.5: sim+sat, 5.5: da+sat, 6.5: sim+da+sat, nan: no flood
    for plot_idx in range(0,23):
        comp_fld(axes[plot_idx],lon_ret1, lon_ret2,lat_ret1, lat_ret2, lon2d_sim1, lat2d_sim1, cal_fld1[:,:,plot_idx], size_fld1[:,:,plot_idx], plot_idx)
    axes[24-1].axis('off')
    for plot_idx in range(0,23):
        comp_fld(axes[plot_idx+24],lon_ret3, lon_ret4,lat_ret3, lat_ret4, lon2d_sim2, lat2d_sim2, cal_fld2[:,:,plot_idx], size_fld2[:,:,plot_idx], plot_idx)
    axes[48-1].axis('off')
    plt.tight_layout()
    plt.show()
    # plt.savefig('/work/a06/yingying/Code/plot/online/jupyter/exp_'+exp+dahour_str+'/'+'2_flood'+yyyymmddhh[0:8]+'.jpg', format='jpg',dpi=600)
    plt.close()
# plot_flood_condition_all()
